# Prepare Hotnet2 data

In [39]:
import pandas as pd
import numpy as np
import os, sys

In [40]:
out_dir = '../data/pancancer/hotnet2/'

## 1. Prepare CNAs and SNVs in Hotnet2 required format

In [41]:
# first, load CNAs to pandas
cnas = pd.read_csv('../data/pancancer/TCGA/cna_samples_syn.tsv', sep='\t')
cnas.set_index('Name', inplace=True)
cnas = cnas.loc[:, cnas.sum(axis=0) > 0] # remove samples without CNAs

# then, load SNVs as well
snvs = pd.read_csv('../data/pancancer/TCGA/snv_samples_syn.tsv', sep='\t')
snvs.set_index('Hugo_Symbol', inplace=True)

In [43]:
with open(os.path.join(out_dir, 'cna_makeheat_input.txt'), 'w') as f:
    for sample in cnas.columns:
        aff_genes = cnas[cnas[sample] == 1].index
        # HACKY: Treating all CNAs here as AMPs because I lost track of what they were...
        row = sample + '\t' + '(A)\t'.join(aff_genes) + ' (A)\n'
        f.write(row)

with open(os.path.join(out_dir, 'snv_makeheat_input.txt'), 'w') as f:
    for sample in snvs.columns:
        aff_genes = snvs[snvs[sample] > 0].index
        row = sample + '\t' + '\t'.join(aff_genes) + '\n'
        f.write(row)

## 2. Run Hotnet2 script to generate heat file
Use the following command on python2, assuming the hotnet2 directory to be located in the same directory as the GCN (legionella):

**python makeHeatFile.py mutation --cna_file ../legionella/data/cancer/hotnet2/cna_makeheat_input.txt --snv_file ../legionella/data/cancer/hotet2/snv_makeheat_input.txt -o heat_syn_cnasnv.json**

For the network, use:

**python makeNetworkFiles.py -e ../legionella/data/cancer/hotnet2/networks/irefindex9/irefindex9_edge_list -i ../legionella/data/cancer/hotnet2/networks/irefindex9/irefindex9_index_gene -nn irefindex9 -p irefindex9 -b 0.45 -o network/ -c 20**

## 3. Run Hotnet2